In [23]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from model.reasoning import ConnectionsDataModule
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_MATH_CONFIG, GRPO_CONNECTIONS_CONFIG
from trl_wrapper.wrapper_config import DatasetConfig
from IPython.display import display

wrapper = TrainerWrapper(GRPO_CONNECTIONS_CONFIG)

wrapper.init_data_module()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2025-03-27 20:26:58.877 | INFO     | trl_wrapper.trainer_wrapper:__init__:266 - Setting padding side to: left
2025-03-27 20:26:58.901 | INFO     | trl_wrapper.wrapper_config:__init__:133 - Cache dir: ../dataset_caches/connections_data_module
Map: 100%|██████████| 353/353 [00:00<00:00, 16569.74 examples/s]


In [25]:
# sample = data_module.train_dataset[0]
# sample
wrapper.data_module.train_dataset
first_sample = wrapper.data_module.train_dataset[0]

conv = first_sample['prompt']
fake_thoughts = "<reasoning>\nFirst four are academic\n</reasoning>\n"
fake_response = {'role': "assistant", 'content': fake_thoughts + first_sample['answer'] + "\n"}
# conv = [*conv, {'role': "assistant", 'content': first_sample['answer']}]

In [26]:
# print(fake_response['content'])
display(first_sample['answer_formatted'])

[['bass', 'harp', 'horn', 'organ'],
 ['arch', 'ball', 'sole', 'toe'],
 ['come', 'down', 'sit', 'stay'],
 ['dog', 'heel', 'jerk', 'snake']]

In [36]:
for reward_func in wrapper.data_module.reward_functions():
    rew = reward_func([conv], [[fake_response]], answer=first_sample['answer_formatted'])

2025-03-27 20:41:39.915 | INFO     | model.reasoning:xmlcount_reward_func:59 - XML count rewards: [0.25]
2025-03-27 20:41:39.916 | INFO     | model.reasoning:strict_format_reward_func:105 - Strict format rewards: [0.0]
2025-03-27 20:41:39.916 | INFO     | model.reasoning:connections_reward_func:174 - Connections scores: [4.0]
